In [2]:
import pyspark

pyspark.__version__

'3.3.1'

In [62]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [64]:
spark_df = spark.read.csv(
    path="data/random-generator.csv",
    header=True,
    quote='\"',
    escape='\"')
print(spark_df.count(), len(spark_df.columns))

10 4


In [65]:
spark_df.dtypes

[('Name', 'string'),
 ('Email', 'string'),
 ('Phone', 'string'),
 ('Description', 'string')]

In [66]:
spark_df.show(5, truncate=False)

+-----------------------------+----------------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Name                         |Email                 |Phone          |Description                                                                                                                                                                                      |
+-----------------------------+----------------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|"Miss Beatrice Murazik\u7684"|wcole@harber.com      |425-800-0568   |Quod consectetur laboriosam voluptate cupiditate tempore harum autem ut. Ut in sit accusantium facilis ea dolores. Molestias rerum aut 

##### Unicode

In [67]:
import unicodedata
import ast

print(type(ast.literal_eval('''['test',1]''')))
print(type(ast.literal_eval('''{'test':1}''')))
print(unicodedata.normalize('NFC', '\u7684'))
print(repr('\u7684'))

<class 'list'>
<class 'dict'>
的
'的'


##### Test Dictionary / Struct

In [68]:
import pandas as pd

pdf = pd.DataFrame({
    "foo": [1, 2, 3],
    "bar": [{"a": 1, "b": 10}, {"a": 2, "b": 20}, {"a": 3, "b": 30}]
})

df = spark.createDataFrame(pdf)
df.coalesce(1).write.mode("overwrite").parquet("output/test.parquet")

In [70]:
read_df = spark.read.parquet("output/test.parquet")

print(read_df.count(), len(read_df.columns))
print(read_df.printSchema())
read_df.show(5, truncate=False)

3 2
root
 |-- foo: long (nullable = true)
 |-- bar: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)

None
+---+-----------------+
|foo|bar              |
+---+-----------------+
|1  |{a -> 1, b -> 10}|
|2  |{a -> 2, b -> 20}|
|3  |{a -> 3, b -> 30}|
+---+-----------------+

